<a href="https://colab.research.google.com/github/saivenkatreddy29/Implementation-of-Hands-on-Large-Language-models-book/blob/main/Jaylamar_book_chapter_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets -q

In [8]:
!pip install -U transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 45.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2


In [38]:
!pip install chatTemplate -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 17.4 MB/s eta 0:00:00


In [41]:
!pip install transformers>=4.28 -q

In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [42]:
from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import ChatTemplate

template_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")


chat_template = ChatTemplate(
    system_message="You are a helpful assistant.",
    user_message="<USER>: {message}",
    assistant_message="<ASSISTANT>: {message}",
)
template_tokenizer.chat_template = chat_template


def format_prompt(example):

  chat = example['messages']
  prompt = template_tokenizer.apply_chat_template(chat,tokenize = False, template = chat_template)
  return {'text':prompt}

dataset = (
    load_dataset("HuggingFaceH4/ultrachat_200k",  split="test_sft")
    .shuffle(seed = 42)
    .select(range(3_000))
)

dataset = dataset.map(format_prompt)

ImportError: cannot import name 'ChatTemplate' from 'transformers' (/usr/local/lib/python3.10/dist-packages/transformers/__init__.py)

In [6]:
print(dataset['text'][2576])

<|user|>
Given the text: Knock, knock. Who’s there? Hike.
Can you continue the joke based on the given text material "Knock, knock. Who’s there? Hike"?</s>
<|assistant|>
Sure! Knock, knock. Who's there? Hike. Hike who? Hike up your pants, it's cold outside!</s>
<|user|>
Can you tell me another knock-knock joke based on the same text material "Knock, knock. Who's there? Hike"?</s>
<|assistant|>
Of course! Knock, knock. Who's there? Hike. Hike who? Hike your way over here and let's go for a walk!</s>



# Models Quantization

In [4]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.9 MB/s eta 0:00:00


In [7]:
!python -c "import bitsandbytes as bnb; print(bnb.__version__)"


0.45.0


In [6]:
!pip install -U bitsandbytes -f  --index-url https://download.pytorch.org/whl/cu118

Looking in links: --index-url
  ERROR: Cannot unpack file /tmp/pip-unpack-73llnv5n/cu118.html (downloaded from /tmp/pip-req-build-bf1ukjyj, content-type: text/html); cannot detect archive format
ERROR: Cannot determine archive format of /tmp/pip-req-build-bf1ukjyj


In [14]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = 'float16',
    bnb_4bit_use_double_quant = True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = 'auto',

    quantization_config = bnb_config,

)

model.config.use_cache = False
model.config.pretrained_tp = 1

#Load LLama tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = 'left'




In [15]:
device

'cuda'

Configuration

LoRA configuration

In [16]:
from peft import LoraConfig,prepare_model_for_kbit_training, get_peft_model

peft_config = LoraConfig(
    lora_alpha = 32,
    lora_dropout = 0.1,
    r = 64,
    bias = 'none',
    task_type = 'CAUSAL_LM',
    target_modules = # target layers
    ['k_proj','gate_proj','v_proj','up_proj','q_proj','down_proj']
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model,peft_config)

In [22]:
from accelerate import Accelerator
accelerator = Accelerator()

model = accelerator.prepare(model)


Training Configuration

In [29]:
from transformers import TrainingArguments

output_dir = "./results"

training_arguments = TrainingArguments(
    output_dir = output_dir,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    optim ="paged_adamw_32bit",
    learning_rate = 2e-4,
    lr_scheduler_type = "cosine",
    num_train_epochs = 1,
    logging_steps = 10,
    fp16 = True,
    gradient_checkpointing = True
)


In [30]:
!pip install trl -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 17.3 MB/s eta 0:00:00


In [36]:
from trl import SFTTrainer

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    max_seq_length=512,

    # Leave this out for regular SFT
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating